# HW-aware Synthesis of MCX

This example shows that implementation of multiple control-x (MCX) logic, using the Classiq synthesis engine, yields different circuit results for different quantum hardware.

The fictitious hardware created here demonstrates how to insert your own custom-designed machine. For comparison, create two types of hardware with `cx, u` basis gates. The difference between them manifests in the connectivity map: one has linear connectivity while the other has all-to-all connectivity.

In [ ]:
from classiq import (
    CustomHardwareSettings,
    Output,
    QArray,
    QBit,
    X,
    allocate,
    control,
    create_model,
    qfunc,
    show,
    synthesize,
    write_qmod,
)
from classiq.synthesis import set_constraints, set_preferences

# define the hardware parameters
max_width = 18

linear_connectivity = [[qubit, qubit + 1] for qubit in range(max_width - 1)]


# define MCX quantum function
@qfunc
def my_mcx(cntrl: QArray[QBit], target: QBit) -> None:
    control(cntrl, lambda: X(target))


# define the MCX parameters within the quantum 'main' function
@qfunc
def main(cntrl: Output[QArray[QBit]], target: Output[QBit]) -> None:
    allocate(15, cntrl)
    allocate(1, target)
    my_mcx(cntrl, target)


# build a model
qmod = create_model(main)
# define synthesis engine constraints
qmod = set_constraints(qmod, optimization_parameter="depth", max_width=max_width)

# define models with different preferences
qmod_linear = set_preferences(
    qmod,
    custom_hardware_settings=CustomHardwareSettings(
        basis_gates=["cx", "u"],
        connectivity_map=linear_connectivity,
    ),
    random_seed=-1,
)
qmod_all_to_all = set_preferences(
    qmod,
    custom_hardware_settings=CustomHardwareSettings(basis_gates=["cx", "u"]),
    random_seed=-1,
)

# write models to files
write_qmod(qmod_linear, "hardware_aware_mcx_linear")
write_qmod(qmod_all_to_all, "hardware_aware_mcx_all_to_all")


# synthesize to create quantum programs and view circuits:
qprog_linear = synthesize(qmod_linear)
show(qprog_linear)

qprog_all_to_all = synthesize(qmod_all_to_all)
show(qprog_all_to_all)

Opening: https://platform.classiq.io/circuit/2vML2x1DIYkGkWEbMlmL9P7g1VV?login=True&version=0.74.0
Opening: https://platform.classiq.io/circuit/2vML3zpRNBgFq7tBJxCfxUZJjjj?login=True&version=0.74.0


Comparison of the two circuits shows that applying MCx using different connectivity maps yields different implementation.

Using "all-to-all" connectivity, the synthesis engine chooses as the best implementation a recourse based on "Maslov2015" [[1](#Maslov)] that was written on the Classiq platform. Using that, the manufactured circuit has 18 qubits; i.e., it uses two auxiliary qubits. The total depth of the circuit is:

In [9]:
print(qprog_all_to_all.transpiled_circuit.depth)

378


When using linear connectivity, the best implementation chosen by the synthesis engine is, in fact, different: an algorithm developed by Classiq, which is better suited for this map. Here, the manufactured circuit uses 18 qubits with only one auxiliary and has a depth of:

In [10]:
print(qprog_linear.transpiled_circuit.depth)

781


## References

<a id='Maslov'>[1]</a>: [Maslov, D., 2016. Advantages of using relative-phase Toffoli gates with an application to multiple control Toffoli optimization. Physical Review A, 93(2), p.022311.](https://arxiv.org/pdf/1508.03273.pdf)
